# Homology on neighborhoods
In this notebook we evaluate betti numbers on a two step neighborhood on the nodes of a graph.

Our final goal is to give the compute 1-betti numbers of the neighoborhoods of the nodes of graphs extracted at different levels of sparsity values of the Coma Dataset and to compare it with the ones computed on a Watts-Strogatz graph.

To do so, we will train a GNN onto the Watts-Strogatz data and see which value of the rewiring probabilities are predicted when the trained model is applied to the betti numbers computed onto the Coma dataset

## Computing the neighborhood of a node
We now write a function to compute the two-step neighbohoods of a graph 

In [1]:
import networkx as nx

def neighborhood(adj_mat, node):
    graph=nx.from_numpy_matrix(adj_mat)
    nbhd=graph.neighbors(node)
    nbhds=[]
    nodes=[node]
    for n in nbhd:               
        nbhds.append(graph.neighbors(n))
        nodes.append(n)
    for neighbor in nbhds:
        for n in neighbor:
            nodes.append(n)
    nodes=list(set(nodes))
    return graph.subgraph(nodes),nodes


## Computing betti numbers of the neighborhoods of the nodes
We write a function that for each network, each given threshold and each node returns the value of the n-dimensional betti number. The return value will be  list of two dimensional array

In [7]:
from HomologyPackage.NetworkGeneration import Erdos_Renyi, Watts_Strogatz, ring_lattice
from HomologyPackage import adjacency_matrix
from gudhi import SimplexTree
import numpy as np
#these last two are for progress bars
from tqdm.notebook import tqdm
from tqdm.contrib import tenumerate

def betti_neighborhood(net_matrices, sparsity_values, dimension=1):
    betti_numbers=list()
    for i,net in tenumerate(net_matrices,desc='network loop', leave=True):
        betti_numbers.append(np.zeros([len(sparsity_values),len(net)]))
        
        for j,eps in tenumerate(sparsity_values,desc='eps loop', leave=False):
                
            adj_mat= adjacency_matrix(net,eps,method='sparsity_notconnected')
            for node in tqdm(np.arange(len(net)),desc='node loop',leave=False):
                
                nbhd,nbhd_nodes=neighborhood(adj_mat,node)
                
                #construct simplex
                simplex=SimplexTree()
                for edge in list(nbhd.edges):
                    simplex.insert(edge)
                    
                #compute betti number
                
                simplex.expansion(dimension+1)
                simplex.compute_persistence(persistence_dim_max=True)
                try:
                    #print(simplex.betti_numbers())
                    betti_numbers[i][j,node]=simplex.betti_numbers()[dimension]
                except:
                    betti_numbers[i][j,node]=0
                    
    return betti_numbers

seed=0
er = Erdos_Renyi(90,seed=seed)   
b=betti_neighborhood([er], [0.1,0.2,0.3], dimension=2)
print(b)

network loop:   0%|          | 0/1 [00:00<?, ?it/s]

eps loop:   0%|          | 0/3 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

[1, 103, 0]
[1, 136, 0, 0]
[1, 104, 0, 0]
[1, 110, 0, 0]
[1, 122, 0, 0]
[1, 99, 0, 0]
[1, 67, 0]
[1, 73, 0]
[1, 107, 0]
[1, 86, 0]
[1, 86, 0]
[1, 25, 0]
[1, 80, 0, 0]
[1, 73, 0, 0]
[1, 70, 0]
[1, 79, 0, 0]
[1, 56, 0]
[1, 159, 0, 0]
[1, 112, 0, 0]
[1, 132, 0, 0]
[1, 91, 0]
[1, 86, 0]
[1, 87, 0]
[1, 83, 0, 0]
[1, 138, 0, 0]
[1, 27, 0]
[1, 123, 0]
[1, 77, 0]
[1, 88, 0]
[1, 105, 0]
[1, 73, 0, 0]
[1, 129, 0]
[1, 143, 0]
[1, 59, 0, 0]
[1, 93, 0, 0]
[1, 102, 0]
[1, 67, 0]
[1, 130, 0, 0]
[1, 71, 0]
[1, 101, 0]
[1, 126, 0]
[1, 85, 0]
[1, 73, 0, 0]
[1, 49, 0]
[1, 90, 0]
[1, 108, 0, 0]
[1, 37, 0]
[1, 77, 0]
[1, 55, 0, 0]
[1, 79, 0]
[1, 53, 0]
[1, 51, 0]
[1, 57, 0, 0]
[1, 70, 0]
[1, 85, 0, 0]
[1, 73, 0]
[1, 113, 0, 0]
[1, 116, 0, 0]
[1, 57, 0]
[1, 88, 0]
[1, 118, 0, 0]
[1, 75, 0]
[1, 69, 0]
[1, 61, 0]
[1, 31, 0]
[1, 66, 0, 0]
[1, 55, 0]
[1, 75, 0, 0]
[1, 81, 0]
[1, 127, 0, 0]
[1, 81, 0]
[1, 37, 0]
[1, 69, 0]
[1, 65, 0]
[1, 119, 0]
[1, 43, 0]
[1, 108, 0]
[1, 71, 0]
[1, 139, 0, 0]
[1, 128, 0]
[1, 53

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

[1, 26, 71, 5]
[1, 28, 74, 5]
[1, 29, 71, 5]
[1, 30, 73, 5]
[1, 24, 71, 5]
[1, 30, 70, 5]
[1, 27, 58, 2]
[1, 27, 58, 5]
[1, 28, 74, 5]
[1, 26, 72, 5]
[1, 28, 74, 5]
[1, 29, 73, 5]
[1, 28, 72, 5]
[1, 25, 72, 5]
[1, 28, 74, 5]
[1, 30, 69, 5]
[1, 24, 62, 5]
[1, 28, 74, 5]
[1, 31, 70, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 28, 72, 5]
[1, 26, 74, 5]
[1, 34, 56, 1]
[1, 28, 74, 5]
[1, 27, 39, 4]
[1, 27, 71, 5]
[1, 27, 70, 5]
[1, 27, 68, 5]
[1, 28, 66, 5]
[1, 28, 42, 4]
[1, 27, 65, 5]
[1, 28, 74, 5]
[1, 27, 70, 5]
[1, 29, 61, 5]
[1, 32, 54, 4]
[1, 28, 67, 5]
[1, 28, 74, 5]
[1, 29, 64, 5]
[1, 26, 74, 5]
[1, 28, 74, 5]
[1, 26, 74, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 27, 63, 5]
[1, 27, 74, 5]
[1, 23, 52, 5]
[1, 26, 73, 5]
[1, 31, 61, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 29, 52, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 28, 74, 5]
[1, 29, 72, 5]
[1, 31, 65, 5]
[1, 27, 71, 5]
[1, 28, 73, 5]
[1, 35, 54, 3]
[1, 28, 74, 5]
[1, 28, 60, 5]
[1, 29, 54, 3]
[1, 28, 70, 5]
[1, 29, 67

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 477, 181]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 188]
[1, 0, 488, 18

In [4]:
seed = 0
er = Erdos_Renyi(90,seed=seed)
ws01 = Watts_Strogatz(90,0.01,seed=seed)
ws1 = Watts_Strogatz(90,0.1,seed=seed)
ws5 = Watts_Strogatz(90,0.5,seed=seed)
rl= ring_lattice(90,seed=seed)

b=betti_neighborhood([rl,ws01,ws1,ws5,er],np.linspace(0.01,0.34,10))
b

network loop:   0%|          | 0/5 [00:00<?, ?it/s]

eps loop:   0%|          | 0/10 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

...

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.,

### Betti number of Coma dataset in the small world regime
Now wi will compute the betti numbers of the neighborhoods of the coma dataset. We choose to take 20 equally spaced sparsity values in the small world regime [0.05,0.34] of costs

In [5]:
import os

path="regional-differentiation-based-on-graph-nodal-statistics-for-functional-brain-connectivity-networks-characterization/DATA/cor_mat_coma"
files=os.listdir(path) #make a list of all the files' names at the path 

In [6]:
patient_files = []
control_files = []
for file in files:
    if 'Patient' in file:
        patient_files.append(file)
    else:
        control_files.append(file)
        
patients = [np.loadtxt(path+"/"+file, usecols=range(1,91), skiprows=1, dtype='float') for file in patient_files]
controls = [np.loadtxt(path+"/"+file, usecols=range(1,91), skiprows=1, dtype='float') for file in control_files]

In [7]:
sparsity_values=np.linspace(0.05,0.34,20)

nbhd_betti_patients=betti_neighborhood(patients,sparsity_values)
nbhd_betti_controls=betti_neighborhood(controls,sparsity_values)

import pickle

with open('data/patients_nbhd_betti', "wb") as f:
    pickle.dump((nbhd_betti_patients,patient_files,sparsity_values), f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data/controls_nbhd_betti', "wb") as f:
    pickle.dump((nbhd_betti_controls,control_files,sparsity_values), f, protocol=pickle.HIGHEST_PROTOCOL)
    

network loop:   0%|          | 0/16 [00:00<?, ?it/s]

eps loop:   0%|          | 0/20 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

...

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

We want to compare these values with the ones computed on the Watts-Strogatz random graphs. Therefore we simulate 3000 correlation matrices for values of the rewiring probability between 0 and 0.5.

In [9]:
probs=[]
ws_corr_mats=[]
for i,p in tenumerate(np.linspace(0,0.5,100)):
    for seed in range(1):
        probs.append(p)
        ws_corr_mats.append(Watts_Strogatz(90,p,seed=101*seed*i[0]))

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
nbhd_betti_ws=betti_neighborhood(ws_corr_mats,sparsity_values)

network loop:   0%|          | 0/100 [00:00<?, ?it/s]

eps loop:   0%|          | 0/20 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

...

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

node loop:   0%|          | 0/90 [00:00<?, ?it/s]

We finally save the results on some files.

In [11]:
with open('data/ws_nbhd_betti', "wb") as f:
    pickle.dump((nbhd_betti_ws,ws_corr_mats,probs,sparsity_values), f, protocol=pickle.HIGHEST_PROTOCOL)
       